In [82]:
"""
Andreas Burger
Exam ID: 37
"""

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import (BatchNormalization, Conv2D, Conv2DTranspose, Dense,
                                            Dropout, Flatten, Multiply, Add, Input,
                                            Conv2D, Activation, BatchNormalization, MaxPool2D,
                                            UpSampling2D, Conv2DTranspose,
                                            LeakyReLU, Reshape)
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import random_normal
from tensorflow.keras.utils import to_categorical
import time
#import tensorflow_probability as tfp
#tf.compat.v1.enable_eager_execution()


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [77]:
# 3

"""
How can the hidden dense layer be modified such that the weights are initialised
with a Gaussian with mean zero and standard deviation 100? Provide your answer in python
code which in principle can be executed.
"""

initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=100.)
layer = Dense(3, kernel_initializer=initializer)



In [78]:
# 5

"""
(9 points) Write down an autoencoder using Keras which can be used for 2D Ising spin configu-
rations (40x40). Your network has to have 100 000 trainable parameters. Submit your network
as an executable code. (Hint: Layers without a bias might render matching the exact number
of parameters easier.)
"""
"""
autoencoder = Sequential()
autoencoder.add(Flatten)
autoencoder.add(Dense(100,  activation='elu', use_bias=False, input_shape=(40*40)))
autoencoder.add(Dense(100,  activation='elu', use_bias=False))
autoencoder.add(Dense(10,    activation='linear', use_bias=False, name="bottleneck"))
autoencoder.add(Dense(100,  activation='elu', use_bias=False))
autoencoder.add(Dense(100,  activation='elu', use_bias=False))
autoencoder.add(Dense(40*40,  activation='sigmoid', use_bias=False))
autoencoder.compile(loss='mean_squared_error', optimizer = Adam())
"""


input_network = Input(shape=(40,40))
x1 = Flatten()(input_network)
x = Dense(10, activation='relu', use_bias=False)(x1)
x = Dense(1000, activation='relu', use_bias=False, name="afterinput")(x)
x = Dense(10, activation='linear', use_bias=False, name="bottleneck")(x)
x = Dense(100, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(100, activation='relu', use_bias=False)(x)
x = Dense(400, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
x = Dense(10, activation='relu', use_bias=False)(x)
output = Dense(40*40, activation='softmax', use_bias=False)(x)

autoencoder = Model(input_network,outputs=output, name="autoencoder")


autoencoder.summary()


Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_65 (InputLayer)        [(None, 40, 40)]          0         
_________________________________________________________________
flatten_59 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_471 (Dense)            (None, 10)                16000     
_________________________________________________________________
afterinput (Dense)           (None, 1000)              10000     
_________________________________________________________________
bottleneck (Dense)           (None, 10)                10000     
_________________________________________________________________
dense_472 (Dense)            (None, 100)               1000      
_________________________________________________________________
dense_473 (Dense)            (None, 10)                

In [79]:
# 6 GAN

"""
Provide an executable python implementation of the training step where you make
two gradient updates on the training data for the generator when the generator is only updated
once.
"""
latent_dim = 10
Batch_Size = 50

generator = Sequential(name="generator")
generator.add(Dense(10, use_bias=False, input_shape=(latent_dim,)))

discriminator = Sequential(name="discriminator")
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    gl = cross_entropy(tf.ones_like(fake_output), fake_output)
    return gl

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def train_step(images):
    latent_vector = tf.random.normal([Batch_Size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(latent_vector, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


    # second time for generator only
    latent_vector = tf.random.normal([Batch_Size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(latent_vector, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      #disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    #gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    #discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [80]:
# 7
"""
Implement a custom loss function which ensures that the following differential equa-
tion is satisfied x' = squrt(1 + x^2) * x
"""

inp = Input(shape=(1,))
x = Dense(100,activation='tanh')(inp)
x = Dense(100,activation='tanh')(x)
out = Dense(1)(x)

model = Model(inputs=inp,outputs=out)


train_data = np.random.normal(loc=0, scale=1, size=100) # gaussian
train_data /= train_data.max()  # rescale to [-1,1] interval
x = tf.Variable(train_data)

with tf.GradientTape() as t3:
    with tf.GradientTape() as t1:
        u = model(x ,training=True)
    du_dx = t1.gradient(u,x)

    loss = (du_dx - (np.sqrt(1 + u**2) * u) )**2


InvalidArgumentError: Matrix size-incompatible: In[0]: [1,100], In[1]: [1,100] [Op:MatMul]

In [81]:
"""
Write a training step which uses this custom loss function evaluated with 1000
uniformly sampled points in the interval (-1,1) for training.
"""
optimizer = tf.keras.optimizers.Adam(1e-4)


inp = Input(shape=(1,))
x = Dense(100,activation='tanh')(inp)
x = Dense(100,activation='tanh')(x)
out = Dense(1)(x)

model = Model(inputs=inp,outputs=out)
#model.summary()

def train_step(samples):
    train_data = np.random.uniform(-1,1,size=(samples,1))
    #train_data /= train_data.max()  # rescale to [-1,1] interval
    x = tf.Variable(train_data)

    with tf.GradientTape() as t3:
        with tf.GradientTape() as t1:
            u = model(x ,training=True)
        du_dx=t1.gradient(u,x)

        # u''(x) + a u'(x) + b u(x) = 0
        #loss = (d2u_dx + a*du_dx + b*u)**2

        loss = (du_dx - (np.sqrt(1 + u**2) * u) )**2


    gradients_of_nn = t3.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients_of_nn, model.trainable_variables))

    print(np.mean(loss))

In [ ]:
"""
Write a training loop where this training step is applied 100 times and output the
training error for each training step.
"""

def train(epochs = 1):
    for epoch in range(epochs):
        train_step(100)

#train()